In [ ]:
knitr::include_graphics("https://slcladal.github.io/images/acqva.jpg")



In [ ]:
# load packages for website
library(knitr) 
library(kableExtra) 
library(DT)


# Introduction

During the workshop on mixed-effects modeling, we talked about (multi-)collinearity and [Jorge González Alonso](https://en.uit.no/ansatte/person?p_document_id=473118) asked if collinearity reflected shared variance (what I thought) or predictability of variables (what Jorge thought).

Well, we are both right.

***
>(Multi-)collinearity is defined as the correlation between predictors. (Multi-)collinearity is a problem because which predictor is reported as being significant depends on the presence of other predictors in the model.

***

Because (multi-)collinearity represents a correlation between predictors, it reflects the predictability of predictors based on the values of other predictors. This correlation or predictability is a problem because when predictors correlate, the variance that they explain overlaps which results in unstable models. The instability occurs because what the model reports as significant depends on the presence of other predictors. 

To test this, I generate a data set with 4 independent variables `a`, `b`, `c`, and `d`. And the data set contains two potential response variables `r1` (which is random) and `r2` (where the first 50 data points are the same as in `r1` but for the second 50 data points I have added 50 to the data points 51 to 100 from `r1`). This means that the predictors a and d should both strongly correlate with `r2` an effect for 


In [ ]:
# load packages
library(tidyverse)
library(rms)
# create data set
# responses
# 100 random numbers
r1 <- rnorm(100, 50, 10)
# 50 smaller + 50 larger numbers
r2 <- c(r1[1:50], r1[51:100] + 50)
# predictors
a <- c(rep("1", 50), rep ("0", 50))
b <- rep(c(rep("1", 25), rep ("0", 25)), 2)
c <- rep(c(rep("1", 10), rep("0", 10)), 5)
d <- c(rep("1", 47), rep ("0", 3), rep ("0", 47), rep ("1", 3))
# create data set
df <- data.frame(r1, r2, a, b, c, d)


In [ ]:
# inspect data
df %>%
  head(10) %>%
  kable(caption = "First 10 rows of df data.") %>%
  kableExtra::kable_styling(bootstrap_options = c("striped", "hover", "condensed", "responsive"), 
                            full_width = F)


Here are the visualizations of r1 and r2



In [ ]:
plot(r1, pch = 20)
plot(r2, pch = 20)


# Fit first model

Now, I fit a first model. As the response is random, we do not expect any of the predictors to have a significant effect and we expect the R^2^ to be rather low.


In [ ]:
m1 <- lm(r1 ~ a + b + c + d, data = df)
# inspect model
summary(m1)


We now check for (multi-)collinearity using the `vif` function from the `rms` package. Variables `a` and `d` should have high variance inflation factor values (vif-values) because they overlap very much!



In [ ]:
# extract vifs
rms::vif(m1)


Variables `a` and `d` do indeed have high vif-values.

We now fit a second model to the response which has higher values for the latter part of the response. Both `a` and `d` strongly correlate with the response. **But** because `a` and `d` are collinear, `d` should not be reported as being significant by the model. The R^2^ of the model should be rather high (given the correlation between the response r2 and `a` and `d`).


In [ ]:
m2 <- lm(r2 ~ a + b + c + d, data = df)
# inspect model
summary(m2)


As predicted, `d` is not  reported as being significant by the model. We look at a model without `a` which should cause `d` to be significant.



In [ ]:
m3 <- lm(r2 ~ b + c + d, data = df)
# inspect model
summary(m3)


As we suspected, now, `d` is reported as being significant by the model. We return to `m2` and extract the vif-values.



In [ ]:
# extract vifs
rms::vif(m2)


The vif-values are identical which shows that what matters is if the variables are predictable. To understand how we arrive at vif-values, we inspect the model matrix.



In [ ]:
# inspect model matrix
mm <- model.matrix(m2)


In [ ]:
# inspect data
mm %>%
  head(15) %>%
  kable(caption = "First 15 rows of the model matrix.") %>%
  kableExtra::kable_styling(bootstrap_options = c("striped", "hover", "condensed", "responsive"), 
                            full_width = F)


We now fit a linear model in which we predcit `d` from the other predictors in the model matrix.



In [ ]:
mt <- lm(mm[,5] ~ mm[,1:4])
summary(mt)$r.squared


The R^2^ shows that the values of `d` are explained to 78.4 percent by the values of the other predictors in the model.

Now, we can write a function (taken from @gries2021statistics) that converts this R^2^ value into a vif-value.


In [ ]:
R2.to.VIF <- function(some.modelmatrix.r2) {
return(1/(1-some.modelmatrix.r2)) } 
R2.to.VIF(0.784)


The function outputs the vif-value of `d`. This shows that the vif-value of `d` represents its predictability from the other predictors in the model matrix and not the amount of shared variance (as I thought).

# Citation & Session Info 

Schweinberger, Martin. `r format(Sys.time(), '%Y')`. *What does multicollinearity mean?*. Tromsø: The Artic University of Norway. url: https://slcladal.github.io/mcol.html (Version `r format(Sys.time(), '%Y.%m.%d')`).


In [ ]:
@manual{schweinberger`r format(Sys.time(), '%Y')`mmws,
  author = {Schweinberger, Martin},
  title = {What does multicollinearity mean?},
  note = {https://slcladal.github.io/mcol.html},
  year = {2021},
  organization = "Arctic University of Norway, AcqVA Aurora Center},
  address = {Tromsø},
  edition = {`r format(Sys.time(), '%Y.%m.%d')`}
}


In [ ]:
sessionInfo()



***

[Back to top](#introduction)

***

# References
